# SUMMARY

## 1. EDA

First I loaded the datasets and inspected the data. There were 3 primary datasets
1. `credit_card_transactions-ibm_v2.csv` - a data set with roughly 24 million rows
2. `sd254_users` - A dataset of all the users that made transactions with some datapoints associated to each
3. `sd254_cards` - A dataset of all the different types of cards associated to each user

I would go ahead to clean and join these datasets together and get all relevant datapoints into one dataset. The cleaning process consisted of:

- Renaming columns
- removing '$'  from amounts
- Creating required date fields 
- Filling null values
- Converting binary string columns into booleans
- Creating new features that I thought would be helpful in my analysis.

I would go on to look at each feature individually besased upon only overall and fraud transaction tendcies. There were some surprising trends when comparing the two sets. The initial columns that stuck out to me were
1. `year`
2. `month`
3. `day_of_week`
4. `hour`
5. `use_chip`

Not surprising, the merchant fields all showed a simliar patterns, with speicifc merchants and industries leading the top of fraud cases. The distribution of these were top heavy with the same offender, this can be mostly related to online transactions. These included
1. `merchant_name`
2. `merchant_city`
3. `merchant_state`
4. `zip`
5. `mcc`

Surpringly `card_type` and `has_chip` did not show any type of pattern for overall and fraud transactions. My initial reaction would be that prepaid cards would have been most likely for fraud.

`errors` was a interesting feature. Most of our fraud did not contain an error, however neither did most of our transactions. I thought this would be a good option to test in our models and not just remove

These were mostly what were included in our dataset, however I went on the build my own features. Here are all the ones I added to the initial dataset
1. `same_state_flag` - Did the transaction occur in the same state where the card owner resides
2. `us_state` - Is the state an actual US state
3. `user_transaction_count` - Total number of user transactions
4. `average_amount` - Average transaction amount for each user
5. `total_amount` - Total amount of transactions for each user
6. `amount_change` - The % change from the previous transaction to the next
7. `dti` - Debt to Income ratio 
8. `itpci` - Income to Per Capita Income

The `same_state_flag` and `us_state were roughly` the same in counts, I decided to leave out `us_state`. These were highly correlated with fraud so I thought it was good to have one.

The rest of my made up columns yielded little information about fraud. After thinking through the problem, most of these items were representive of the final state of the customer, not the point in time of the transaction. Either they werent uptodate or they were features built on information not available at the time (ie total transactions when all their transactions hadn't been made). This could have been counteracted with using to date functions, such as transactions_to_date. I ultimately decided to leave this features out and move on to modeling due to time constraints.

My final feature set after EDA consisted of:

```python
['user', 'year', 'month', 'day_of_week', 'is_weekend', 'hour_of_day', 'amount', 'amount_bin', 
 'use_chip', 'merchant_name', 'merchant_city', 'merchant_state', 'zip', 'mcc', 'card_type', 
 'has_chip', 'errors', 'same_state_flag', 'us_state', 'user_transaction_count', 'merchant_count', 
 'total_amount', 'average_amount', 'days_since_last_transaction', 'amount_change', 'is_fraud']

Next I decided to run a random forest model on these and get feature importance. I did a base line random forest and then a grid searched random forest for optimization, both returned rough the same results. I arbitraily picked the top 10 features based upon importance and used these to contruct my model.

## Modeling

In the model.ipynb, I ran some initial models to see what types of results I would get. The results were less than ideal and really made me take a tough look at my EDA and features. Much more were included after these, but it did give me a good baseline. 

After adding my new features into a random forest model, the results werent much better than my model.ipynb results. However I decided on trying xgboost for my model, which produced much better results with a randomized grid search for parameter tuning. I went with the randomzied grid search to improve the speed of training. I also decided to train this model on a sample of 1 million samples and test on 1 million samples. This was primarly due to lack of time to train the model.

I ended up with the following results from the model
1. Accuracy - 99%
2. Precision - 97%
3. Recall - 70%
4. F1-Score - 81%